## 📥 Fetch data

We'll fetch contents from VS Code docs ([GitHub repo](https://github.com/microsoft/vscode-docs/blob/main)) and save it into a json format that can be used for RAG models.

In [ ]:
# Recursively fetch all md files from a GH repo using gh api

import os
import requests
from dotenv import load_dotenv

load_dotenv()

REPO_OWNER = 'microsoft'
REPO_NAME = 'vscode-docs'

def get_markdown_files_from_github(path=''):
    url = f'https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/contents/{path}'
    headers = {
        'Authorization': f'token {os.getenv("GITHUB_TOKEN")}'
    }
    response = requests.get(url, headers=headers)
    markdown_files = []

    if response.status_code == 200:
        files = response.json()

        for file in files:
            if file['type'] == 'dir':
                markdown_files += get_markdown_files_from_github(file['path'])
            elif file['type'] == 'file' and file['name'].endswith('.md'):
                markdown_files.append(file['path'])

        return markdown_files
    else:
        raise Exception(f"Error fetching repo contents: {response.status_code}")
    
# Test: Print all markdown files in the learn/students folder
print(get_markdown_files_from_github("learn/students"))

In [ ]:
# Get the content of a file using gh api

import base64

def get_file_content(file_path):
    url = f'https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/contents/{file_path}'
    headers = {
        'Authorization': f'token {os.getenv("GITHUB_TOKEN")}'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        file_content = base64.b64decode(response.json()['content']).decode('utf-8')
        title = next(line.replace('# ', '') for line in file_content.split('\n') if line.startswith('#'))
        description = next((line.split(':', 1)[1].strip() for line in file_content.split('\n') if line.lower().startswith('metadescription:')), '')
        content = file_content.split(title, 1)[1].strip()
        content = content.replace('<!-- DOWNLOAD_LINKS_PLACEHOLDER -->', '')
        full_content = f"{description}\n\n{content}"
        return {
            'content': full_content, # rate limit could be exceeded for free tier
            'url': f'https://github.com/{REPO_OWNER}/{REPO_NAME}/blob/main/{file_path}',
        }
    else:
        raise Exception(f"Error fetching file content: {response.status_code}")
    
# Test: Print a sample file content
get_file_content("release-notes/v1_94.md")

In [ ]:
# Iterate over each file in `release-notes` and get the content

docs_files = get_markdown_files_from_github("release-notes")

docs_contents = []
for file_path in docs_files:
    content = get_file_content(file_path)
    docs_contents.append(content)

# Print the first few contents to verify
for doc in docs_contents[:5]:
    print(doc)
    print('---')

## 🧩 Chunk data

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3")
]

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

# Initialize the MarkdownHeaderTextSplitter with the headers to split on
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)

# Split each document content and retain the URL
split_docs = []
for doc in docs_contents:
    split_content = markdown_splitter.split_text(doc['content'])
    for split in split_content:
        split_docs.append({
            'content': split.page_content,
            'url': doc['url'],
        })

# Print the first few split documents to verify
for split_doc in split_docs[:5]:
    print(split_doc)
    print('---')

### 📄 Output as Json

In [ ]:
import json

# Write split_docs_dicts to a JSON file
with open('split_docs_contents.json', 'w') as f:
    json.dump(split_docs, f, indent=4)